# Atividade 01 - Parte 01: Inventário de emissões utilizando Python


#### Alunos: Matheus Henrique Lourenço de Oliveira (19205617) e Mayara Dargas Sousa (20203642)

## Introdução

O presente estudo tem o objetivo de realizar uma estimativa das emissões de Material Particulado (PM), Óxidos de Nitrogênio (NOx), Dóxido de Enxofre (SO2) e Dióxido de carbono (CO2) com base em dados do Relatório de ESG do grupo MOHAWK, o qual a marca Eliane revestimentosfaz parte. Foi utilizado o capítulo 11.7 da AP42, pois o empreendimento trata-se de uma Indústria de Revestimentos Cerâmicos (https://www.epa.gov/sites/default/files/2020-10/documents/c11s07.pdf).

Os valores descritos na tabela seguir apresenta os fatores de emissão fornecidos pela AP-42, os quais foram convertidos de lb/ton para kg/ton, conforme instruções da Ap-42. Para o dióxido de enxofre (SO2), foi adotado o valor citado na referência: "Reference 10. For facilities using raw material with a sulfur content greater than 0.07 percent. The variable S represents the raw material sulfur content (percent). For facilities using raw material with a sulfur content less than or equal to 0.07 percent, use 9.5 S lb/ton to estimate emissions.”

## Fluxograma do processo produtivo

In [57]:
# se sobrar tempo colocar o fluxograma aqui 

## Planilha de fatores de emissões compilada

In [ ]:
import pandas as pd
import os
from IPython.display import display

# Coletando o caminho da pasta que estamos
caminho_atual = os.getcwd()

# Definindo o caminho da pasta dos fatores de emissão
caminho_fe = os.path.join(caminho_atual, 'tabelas', 'FE')

# Indicando o número do capítulo e subcapítulo
capitulo = 11
subcapitulo = 7

# Construindo o nome do arquivo
nome_arquivo = f'chp_{capitulo}_{subcapitulo}.csv'

# Caminho completo do arquivo
caminho_arquivo = os.path.join(caminho_fe, nome_arquivo)

# Abrindo o arquivo utilizando o Pandas
fe = pd.read_csv(caminho_arquivo)

# Estilizando a planilha com cores alternadas para as linhas e colunas
styled_fe = fe.style.set_properties(**{'text-align': 'center'}) \
                     .set_table_styles([
                         {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]},
                         {'selector': 'tr:nth-child(odd)', 'props': [('background-color', 'white')]},
                         {'selector': 'th', 'props': [('background-color', '#e6e6e6')]},
                         {'selector': 'td', 'props': [('background-color', 'white')]}
                     ])

# Exibindo a planilha estilizada na tela
display(styled_fe)

#obs: centralizar a planilha e diminuir casas decimais se possível

## Estimativa da produção: premissas adotadas e valores fornecidos

Conforme dados institucionais de 2021, a Eliane revestimentos produz 40.8 milhões de m² de revestimentos por ano, distribuídos em 6 unidades em todo o Brasil. Assumindo a premissa de que cada unidade produz a mesma quantidade, a unidade de interesse, em Criciúma - SC, produziria aproximadamente 6.8 milhões de m² de revestimento.

Além disso, de modo a estimar este valor em toneladas, foram utilizados os valores de peso e medida do modelo mais vendido da fábrica, o porcelanato Bianco plus NA 60X60 com as seguintes especificações técnicas:

- Peso da caixa: 30,38 kg
- Rendimento: 1,44 m²

Portanto, se: 

30,38 kg ----- 1,44 m² 

   X     ----- 6.8 x 10^6 m²

X = 143.46111 x 10^6 kg = 143461,11 toneladas/ano 

ou então, 

11955,09 toneladas/mês. Também assumindo a premissa de uma escala produtiva uniforma ao longo do ano.

## Plotando os fatores de emissão

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

atv = 143461.11  # A unidade descrita aqui é toneladas. Os processos para determinar este valor estão descritos anteriormente
emis = fe['PM'] * atv

cwd = os.getcwd()

# Somando dataframe
print(emis.sum())

# Criando a figura com Matplotlib
# Subplots com número de linhas igual ao número de poluentes
# sharex significa compartilhar o eixo x
fig, ax = plt.subplots(len(fe.columns) - 1, 1, sharex=True, figsize=(9, 15))

# Loop para preencher o subplot com os gráficos de barra
for ii in range(1, len(fe.columns)):
    print(ii)
    ax[ii - 1].bar(fe['Source'], fe.iloc[:, ii], color=['#3856CF'])
    ax[ii - 1].set_xlabel('Processo', fontsize=8)  # Define o tamanho da fonte para o eixo x
    ax[ii - 1].set_ylabel(fe.columns[ii] + '\n(kg/t)', fontsize=8)  # Define o tamanho da fonte para o eixo y

    # Define o tamanho da fonte para os rótulos dos ticks nos eixos x e y
    ax[ii - 1].tick_params(axis='x', labelsize=8)
    ax[ii - 1].tick_params(axis='y', labelsize=8)

    # Define os rótulos dos ticks nos eixos x
    ax[ii - 1].set_xticks(fe['Source'].index)  # Define a localização dos ticks no eixo x
    ax[ii - 1].set_xticklabels(fe['Source'], rotation=45, ha='right', fontsize=8)  # Define os rótulos dos ticks no eixo x

    # Adicionando linhas de grade
    ax[ii - 1].grid(True, linestyle='--', alpha=0.5)

    # Adicionando rótulos nos topos das barras
    for x, y in zip(fe['Source'], fe.iloc[:, ii]):
        if np.isfinite(y):  # Verifica se o valor de y é finito
            ax[ii - 1].text(x, y + 0.05, f'{y:.2f}', ha='center', fontsize=8)

# Salva a figura na pasta figures
fig.savefig(cwd + '\\figures\\barplot_fatorEmiss.png')

plt.show()


### Qual é o processo do empreendimento que possui maior emissão? analise os fatores de emissão através de figuras, comparando os processos

Não é possível avaliar de forma precisa qual o processo do empreendimento que produz mais poluentes devido a ausência de valores de fatores de emissão para todos os processos mencionados, fazendo com que não seja possível avaliá-los de forma igual. Entretanto, se for levado em consideração apenas a emissão de partículado, o processo que possui a maior emissão ao longo de um ano é a o secador ou "dryer".

# Criando taxa de atividade aleatória com sazonalidade

### Série temporal das emissões mensais do empreendimento

XXXXXXXXX EXPLICAR acho que tem que dividir os valores anterioires por por 12 e premissas adotadas para atividade máxima e min
### Nesta sessão nós criaremos uma taxa de atividade de um empreendimento qualquer que contenha sazonalidade ao longo dos meses do ano. Esta variabilidade também deve conter uma componente aleatória.

Dado que só possuímos um dado anual, a taxa de atividade foi convertida para toneladas por dia, resultando em um valor de aproximadamente 393,04 toneladas por dia. Reconhecendo que a produção não é um valor constante e que a produtividade pode variar de um dia para outro, consideramos dois cenários. No primeiro cenário, a taxa de atividade é calculada estipulando uma faixa de valores a partir de atividades máximas e mínimas, em que foi considerado uma variação de 393,04 ±0,5 ton/dia

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Criando vetor de valores aleatórios
atvMax = 393.54
atvMin = 392.54
n = 1000
atv = (atvMax - atvMin) * np.random.rand(n) + atvMin
#print(atv)

# Exportando os dados para um arquivo .csv
output_folder = 'outputs'
output_file = 'dados_aleatorios.csv'
output_path = os.path.join(output_folder, output_file)

# Criando DataFrame com os dados aleatórios
df = pd.DataFrame({'Atividade': atv})
print(df)

# Salvando o DataFrame como arquivo .csv
df.to_csv(output_path, index=False)

# Criando uma figura com os valores aleatórios
fig, ax = plt.subplots()
ax.plot(atv)
plt.show()


In [ ]:
# Criando vetor de datas
import pandas as pd

# Criar um intervalo de datas
datas = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')

# Criar um DataFrame com as datas
df = pd.DataFrame({'Data': datas})

# Extrair os meses das datas e adicioná-los como uma nova coluna no DataFrame
df['Mes'] = df['Data'].dt.month


#criar um array de 1 a 365 e colocar igual a df['Dia']
df['Dia do Ano'] = df['Data'].dt.dayofyear  

# Mostrar o DataFrame resultante
print(df)



In [ ]:
# Ajustar meus números aleatórios com base no comprimento do vetor de datas

# Criando dataframe com datas e valores de atividade aleatória
import pandas as pd
# Extraindo o número de linhas/número de datas
n = df.shape[0]

# Criando atividade aleatória
atv = (atvMax - atvMin)*np.random.rand(n) + atvMin
df['atividade'] = atv
#print(atv)
print(df)
df.to_csv(r"C:\ENS5173_MayaraDargas\ENS5173-2024.1_MayaraDargasSousa\tabelas\ativ0.csv")


#Criando uma figura com os valores aleatórios
fig, ax = plt.subplots()
ax.plot(datas, atv)
plt.show()


In [ ]:
# Criando componente sazonal

sazonalidade = np.cos((2*np.pi)*df['Dia do Ano']/365)+1 #taxa de revestimento produzida diariamente 
plt.plot(sazonalidade)
df['sazonalidade'] = sazonalidade 
df.to_csv 
print(sazonalidade)

df.to_csv(r"C:\ENS5173_MayaraDargas\ENS5173-2024.1_MayaraDargasSousa\tabelas\ativ.csv")

In [ ]:
#Variablidade com Monte Carlo
import numpy as np

# Gerando variabilidade usando o método Monte Carlo
df['variabilidade'] = np.random.rand(len(df)) * (atvMax - atvMin) + atvMin

# Definindo a produção diária aplicando variabilidade
df['producao_diaria'] =  df['variabilidade'] + df['sazonalidade'] 


print (df)


In [ ]:
# Vamos usar essa coluna para o eixo x no gráfico
# Substitua 'Dia do Ano' pelo nome da coluna correspondente em seu DataFrame, se necessário
fig, ax = plt.subplots(2)

# Plotagem do primeiro gráfico
ax[0].plot(df['Dia do Ano'], df['producao_diaria'])
ax[0].set_title('Produção Anual com Variabilidade')
ax[0].set_xlabel('Dia do Ano')
ax[0].set_ylabel('Produção Diária (ton)')

# Plotagem do segundo gráfico
ax[1].plot(df['atividade']*df['sazonalidade'])
ax[1].set_title('Produção Anual sem Variabilidade')
ax[1].set_xlabel('Dia do Ano')
ax[1].set_ylabel('Produção Diária (ton)')

# Adicionando espaço entre os gráficos
plt.subplots_adjust(hspace=1)

plt.show()


Dado que só possuímos um dado anual, a taxa de atividade foi convertida para toneladas por dia, resultando em um valor de aproximadamente 393,04 toneladas por dia. Reconhecendo que a produção não é um valor constante e que a produtividade pode variar de um dia para outro, consideramos dois cenários. No primeiro cenário, a taxa de atividade é calculada estipulando uma faixa de valores a partir de atividades máximas e mínimas, em que foi considerado uma variação de 393,04 ±0,5 ton/dia.

### Qual a variabilidade temporal da taxa de atividade? estime esta taxa e demonstre a componente sazonal (variabilidade entre meses) e aleatória. Faça figuras e discuta. 

In [ ]:
# Criando subplots para cada processo
fig, axs = plt.subplots(7, 3, figsize=(15, 20))
for idx, (processo, idx_fe) in enumerate(processos):
    for ii in range(1, 4):
        axs[idx, ii-1].plot(df['Dia do Ano'], df['producao_diaria']*fe.iloc[idx_fe, ii])
        axs[idx, ii-1].set_ylabel(fe.columns[ii]+'\n (kg)', fontsize=12)
        axs[idx, ii-1].set_title(processo, fontsize=14)  # Adicionando o título do processo em cada gráfico
plt.tight_layout()
plt.show()

# Adicionando linha para cada mês
for date in pd.date_range(start='2023-01-01', end='2023-12-31', freq='MS'):
    ax.axvline(x=date, color='black', linestyle='--', alpha=0.5)

# Configurando o formato do eixo x para mostrar os meses abreviados
ax.xaxis.set_major_locator(mdates.MonthLocator())  # Coloca um marcador em cada mês
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))  # Mostra os três primeiros caracteres do mês


In [ ]:
import matplotlib.dates as mdates  # Importando o módulo de datas do Matplotlib

# Criando subplots para cada processo
fig, axs = plt.subplots(7, 3, figsize=(15, 20))
for idx, (processo, idx_fe) in enumerate(processos):
    for ii in range(1, 4):
        # Plotando o gráfico com os meses na legenda do eixo x
        ax = axs[idx, ii-1]  # Selecionando o eixo atual
        ax.plot(df['Dia do Ano'], df['producao_diaria']*fe.iloc[idx_fe, ii])
        ax.set_ylabel(fe.columns[ii]+'\n (kg)', fontsize=12)
        ax.set_title(processo, fontsize=14)  # Adicionando o título do processo em cada gráfico
        
        # Adicionando linha para cada mês
        for date in pd.date_range(start='2023-01-01', end='2023-12-31', freq='MS'):
            ax.axvline(x=date, color='black', linestyle='--', alpha=0.5)
        
        # Configurando o formato do eixo x para mostrar os meses abreviados
        ax.xaxis.set_major_locator(mdates.MonthLocator())  # Coloca um marcador em cada mês
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))  # Mostra os três primeiros caracteres do mês

plt.tight_layout()
plt.show()


### Qual a soma das emissões considerando todos os processos? qual a emissão média, mínima e máxima (unidades sempre em gramas por segundo - g/s)

In [ ]:
# Calcular as emissões mensais para cada poluente
emissoes_mensais = {}
for poluente in fe.columns[1:]:
    # Agrupar por mês e somar as emissões diárias
    emissoes_mensais[poluente] = df.groupby('Mes')['aleatorioSazonal'].sum()

# Calcular as emissões totais mensais em g/s para cada poluente
emissoes_totais_g_s = {}
for poluente, emissao_mensal in emissoes_mensais.items():
    emissoes_totais_g_s[poluente] = emissao_mensal * 1000 / (30 * 24 * 3600)  # Convertendo kg para g e dividindo pelo número de segundos em um mês

# Calcular as emissões totais para cada poluente considerando todos os processos
emissoes_totais = {}
for poluente in fe.columns[1:]:
    atv_processo = (atvMax - atvMin) * np.random.rand(len(fe)) + atvMin
    emissoes_totais[poluente] = fe[poluente] * atv_processo

# Calcular a emissão média, mínima e máxima para cada poluente
resultados_emissoes = {}
for poluente, emissao_total in emissoes_totais.items():
    emissao_media = emissao_total.mean()  # Média das emissões totais
    emissao_minima = emissao_total.min()  # Mínima das emissões totais
    emissao_maxima = emissao_total.max()  # Máxima das emissões totais
    resultados_emissoes[poluente] = {
        'Emissão Média (g/s)': emissao_media,
        'Emissão Mínima (g/s)': emissao_minima,
        'Emissão Máxima (g/s)': emissao_maxima
    }

# Exibir os resultados para cada poluente
for poluente, resultados in resultados_emissoes.items():
    print(f"Resultados para o poluente '{poluente}':")
    for chave, valor in resultados.items():
        print(f"{chave}: {valor} ")
    print()  # Adiciona uma linha em branco após cada conjunto de resultados


Referências
- https://mohawkind.com/_pdf/Eliane_2022_ESG_PT.pdf
- https://www.eliane.com/institucional
- https://www.eliane.com/produtos/bianco-plus-na-60x60-sc-8017071 

In [ ]:
somar a varialibilidade a sazonalidade  e não multiplicar, eu tenho que colocar em variação diaária 